In [1]:
import math
#import numpy
import autograd.numpy as np
import random
from autograd import elementwise_grad
from scipy.special import comb
import itertools

In [31]:
import logging
import sys

In [41]:
logger = logging.getLogger()

logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))


In [42]:
logger.handlers


[<StreamHandler stdout (NOTSET)>]

In [4]:
state = np.array([1.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
numcards = 8
ownCards = np.array([0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
factor = 1
bluffEpsilon = 0.1  #remember to implement
callsList = np.array([[-1, -1, -1, -1, -1],
                  [ 0, -1, -1, -1, -1],
                  [ 1, -1,  2, -1, -1],
                  [ 3, -1,  4, -1, -1],
                  [ 5, -1, -1,  6, -1]])

def findCall(num):
    for i in range(callsList.size):
        for j in range(callsList[0].size):
            if callsList[i][j] == num:
                return [i, j]



In [5]:
def oneCardProbabilities(state):
    
    total = np.sum(state)
    oneCardProb = (state+1)/max((total+len(state)),1)
    return oneCardProb

In [6]:
def twotypeCall(state, numcards, countOne, countTwo, ownCards, typeOne, typeTwo):
    """ Output is a number representing probability of individual call
    state vector of size 13
    numcards, countOne, countTwo, typeOne, typeTwo numbers
    ownCards vector of size 13
    
    Gets specific call probability from general matrix
    """
    return twotype(state, numcards, countOne, countTwo, ownCards)[typeOne][typeTwo]

def twotype(state, numcards, countOne, countTwo, ownCards):
    """ Output is a 13x13 matrix 
    state vector of size 13
    numCards, countOne, countTwo numbers
    ownCards vector of size 13
    
    Converts state to 2x13x13 matrix
    """

    prob = oneCardProbabilities(state)
    #probPairs = np.array(np.meshgrid(prob,prob)) # so that it has shape (2,13,13)
    probPairs = np.transpose(np.reshape(np.array(list(map(np.array,itertools.product(prob,prob)))),(state.size,state.size,2)),(2,0,1))
    
             #print(probPairs.shape)
#     probabilities = np.zeroes([len(state), len(state)])
#     for i in range(len(state)):
#         for j in range(len(state)):
#             twopairprob(probPairs[i, j], numcards)
    
    
    #probPairs[0] rows, probPairs[1] columns
    return twotypeprob(probPairs, numcards, countOne, countTwo, ownCards)

def twotypeprob(p, numcards, countCardOne, countCardTwo, ownCards):
    """ Output is a 13x13 matrix. 
    p.shape=2x13x13 >state made into grid representing pairs
    numcards, countCardOne, countCardTwo numbers
    ownCards vector of size 13
    
    Returns matrix representing probabilities of calls
    """
    prob = 0
    for i in range(countCardOne, 5):
        for j in range(countCardTwo, 5): #one of the reshapes is unnecessary
            prob += (comb(numcards, i-ownCards).reshape(ownCards.shape[0], 1) * (p[0])**(i-ownCards).reshape(ownCards.shape[0],1)*\
                     comb(numcards-i+ownCards, j-ownCards).reshape(1,ownCards.shape[0]) * (p[1])**(j-ownCards).reshape(1, ownCards.shape[0])*\
                     (1-(p[0]+p[1]))**(numcards-(i+j-ownCards-ownCards.T)))
                    #col*(identicalcols**col)
                    # row*(identicalrows**row)
                    # 13x13 matrix
    return prob
            

def onetypeCall(state, numcards, count, ownCards, cardType):
    return onetype(state, numcards, count, ownCards)[cardType]
    
def onetype(state, numcards, count, ownCards):
    prob = oneCardProbabilities(state)
    return onetypeprob(prob, numcards, count, ownCards)
    
def onetypeprob(p, numcards, countCard, ownCards):
    prob = 0
    for i in range(countCard, 5):
        prob = prob + comb(numcards, i-ownCards)* p**(i-ownCards) * (1-p)**(numcards-i+ownCards)
    return prob

In [7]:
pgrad = elementwise_grad(twotypeCall)
pograd = elementwise_grad(onetypeCall)

In [87]:
pgrad(state, numcards, 2, 2, ownCards, 0, 4)

array([ 0.00719949, -0.001309  , -0.001309  , -0.001309  ,  0.00719949,
       -0.001309  , -0.001309  , -0.001309  , -0.001309  , -0.001309  ,
       -0.001309  , -0.001309  , -0.001309  ])

In [88]:
pograd(state, numcards, 2, ownCards, 4)

array([-0.00786135, -0.00786135, -0.00786135, -0.00786135,  0.09433614,
       -0.00786135, -0.00786135, -0.00786135, -0.00786135, -0.00786135,
       -0.00786135, -0.00786135, -0.00786135])

In [8]:
def updateParameters(call, state, numcards, ownCards):
    #for i in range(10):
    if(call[1] == 0):
       grad = pograd(state, numcards, call[0],  ownCards, call[2])
    else:
       grad= pgrad(state, numcards, call[0], call[1], ownCards, call[2], call[3])
        #state = state + grad/grad.max()/10
    logger.debug(np.exp(grad))
    logger.debug(grad)
    
    return state*np.exp((grad))

In [90]:

def testUpdate():
    call = np.array([3, 0, 0, 4])
    state= np.array([1.0, 1, 1, 1, 1, 1, 1, 1, 1])
    numcards = 4
    ownCards = np.array([0.0, 0, 0, 0, 0, 0, 0, 0, 0])
    state = updateParameters(call, state, numcards, ownCards)

    print("1 prob:" + str(twotypeCall(state, numcards, 1, 0, ownCards, 0, 4)))
    print("2 prob:" + str(twotypeCall(state, numcards, 2, 0, ownCards, 0, 4)))
    print("3 prob:"+ str(twotypeCall(state, numcards, 3, 0, ownCards, 0, 4)))


In [91]:
testUpdate()

AttributeError: type object 'StreamHandler' has no attribute 'level'

In [92]:
twotypeCall(state, numcards, 3, 2, ownCards, 0, 4)

0.0010667368257667965

In [93]:
pgrad(state, numcards, 3, 2, 0, 4)

TypeError: twotypeCall() missing 1 required positional argument: 'typeTwo'

In [9]:
def makeCall(call, state, numcards, ownCards, calls):  #not quite
    allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]    
    ind = calls[call[0]][call[1]]
    bestCall = [-1, -1, -1, -1, 0]
    for i in range(ind+1, len(allCallProb)):  #call strength
        for j in range(allCallProb[i].shape[0]):  #level within call
            if len(allCallProb[i].shape) > 1:
                for k in range(allCallProb[i][j].shape[0]):
                    prob = allCallProb[i][j][k]
                    if((prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)) and k!=j):
                        
                        newcall = findCall(i)
                        bestCall = [newcall[0], newcall[1], j, k, prob]
                    
            else:
                
                prob = allCallProb[i][j]
                if(prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)):
                    newcall = findCall(i)
                    bestCall = [newcall[0], newcall[1], j, -1, prob]
                    
    currLevelProb = allCallProb[ind]
    
    for i in range(call[2], currLevelProb.shape[0]):
        if len(currLevelProb.shape) > 1:
            for j in range(currLevelProb.shape[1]):
                if(i != call[2] or j > call[3]):
                    prob = currLevelProb[i][j]
                    if((prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)) and i != j):
                        
                        newcall = findCall(ind)
                        
                        bestCall = [newcall[0], newcall[1], i, j, prob]
        else:
            if(i != call[2]):
                prob = currLevelProb[i]
                if(prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)):
                    newcall = findCall(ind)
                    bestCall = [newcall[0], newcall[1], i, -1, prob]
    return bestCall
    

In [10]:
def makeDecision(call, state, numcards, ownCards, calls, numrounds):
    #somewhat unnecessary
    allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]

    ownCall = makeCall(call, state, numcards, ownCards, calls)
    oppCallProb = 0
    if(call[1] == 0):
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]]
    else:
        
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]][call[3]]
    logger.debug("probability: " + str(oppCallProb))
    oppCallProb = (1-oppCallProb)*(1-0.80**(numrounds//2+1))
    logger.debug(oppCallProb)
    if ownCall[4] >= oppCallProb:
        return 1  #call
    else:
        return 0  #bluff

In [96]:
state

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [97]:
x = np.array([1,2])

In [98]:
state

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [99]:
makeCall(np.array([3, 2, 2, 1]), state, numcards, calls)

NameError: name 'calls' is not defined

In [100]:
makeDecision(np.array([3,2,2,1]), state, numcards, calls)

NameError: name 'calls' is not defined

In [101]:
allCallProb

NameError: name 'allCallProb' is not defined

In [102]:
prob = oneCardProbabilities(state)

In [103]:
prob

array([0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308,
       0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308,
       0.07692308, 0.07692308, 0.07692308])

In [11]:
def runGame():
    bluffCalled = 0

    numPlayerCards = 4
    numCpuCards = 4
    deckSize = 13
    factor = 1
    
    
    deck = [i for i in range(deckSize) for j in range(4)]
    random.shuffle(deck)
    playerCards=deck[0:4]
    cpuCards = deck[4:8]
    #playerCards = [12, 0, 6, 3]
    #cpuCards = [9, 0, 10, 9]
    
    state = np.array([1.0 for i in range(deckSize)])
    ownCards = np.array([0 for i in range(deckSize)])
    #ownCards[6] = 1
    #ownCards[7] = 1
    #ownCards[9] = 1
    #ownCards[10] = 1
    
    for i in cpuCards:
        ownCards[i] = ownCards[i]+1
        
    print('These are your cards:', playerCards)
    #print(cpuCards)
    previousProbs = [0]
    numrounds= 0
    while not bluffCalled:
        
        userIn = input('Make your decision:')
        if(userIn == "bluff"):
            bluffCalled = 1
            print(cpuCards)
            print(previousProbs)
            break
        else:
            userIn=userIn.split()
            call= np.array([int(userIn[0]), int(userIn[1]), int(userIn[2]), int(userIn[3])])
            
      
            state= updateParameters(call, state, numPlayerCards, ownCards)
           
            numrounds = numrounds+1
            print("rounds: " + str(numrounds))
            logger.debug(state)
            decision = makeDecision(call, state, numPlayerCards, ownCards, callsList, numrounds)
            if not decision:
                print("Bluff")
                print(makeCall(call, state, numPlayerCards, ownCards, callsList))
                bluffCalled = 1
                print(cpuCards)
                
            else:
                print(makeCall(call, state, numPlayerCards, ownCards, callsList)[0:4])
                previousProbs.append(makeCall(call, state, numPlayerCards, ownCards, callsList)[4])
                
            
        
            
            
    
    

In [105]:
pgrad(state, numcards, 4, 3, 1, 2)

TypeError: twotypeCall() missing 1 required positional argument: 'typeTwo'

In [106]:
3**np.array([1, 2, 3])

array([ 3,  9, 27], dtype=int32)

In [72]:
calls[0].size

5

In [92]:
prob = oneCardProbabilities(state)
np.array(list(map(np.array,itertools.product(prob,prob))))

array([[0.04347826, 0.04347826],
       [0.04347826, 0.06521739],
       [0.04347826, 0.08695652],
       [0.04347826, 0.10869565],
       [0.04347826, 0.13043478],
       [0.04347826, 0.15217391],
       [0.04347826, 0.17391304],
       [0.04347826, 0.19565217],
       [0.04347826, 0.2173913 ],
       [0.06521739, 0.04347826],
       [0.06521739, 0.06521739],
       [0.06521739, 0.08695652],
       [0.06521739, 0.10869565],
       [0.06521739, 0.13043478],
       [0.06521739, 0.15217391],
       [0.06521739, 0.17391304],
       [0.06521739, 0.19565217],
       [0.06521739, 0.2173913 ],
       [0.08695652, 0.04347826],
       [0.08695652, 0.06521739],
       [0.08695652, 0.08695652],
       [0.08695652, 0.10869565],
       [0.08695652, 0.13043478],
       [0.08695652, 0.15217391],
       [0.08695652, 0.17391304],
       [0.08695652, 0.19565217],
       [0.08695652, 0.2173913 ],
       [0.10869565, 0.04347826],
       [0.10869565, 0.06521739],
       [0.10869565, 0.08695652],
       [0.

In [134]:
twotype(state, numcards, 3, 2, ownCards)

array([[7.18950219e-05, 6.23392158e-01, 5.36118909e-01, 4.58983086e-01,
        3.91092877e-01, 6.94962101e-04, 8.64181184e-04, 1.04062577e-03,
        1.22154318e-03],
       [2.30097998e-04, 5.86788684e-01, 5.02862922e-01, 4.28940575e-01,
        3.64107782e-01, 2.21449261e-03, 2.75110885e-03, 3.30960438e-03,
        3.88111767e-03],
       [5.16600830e-04, 5.43934449e-01, 4.64385244e-01, 3.94567414e-01,
        3.33559691e-01, 4.94910668e-03, 6.14228819e-03, 7.38169540e-03,
        8.64731809e-03],
       [9.54491227e-04, 4.97426885e-01, 4.22979340e-01, 3.57881752e-01,
        3.01218046e-01, 9.10035627e-03, 1.12826205e-02, 1.35447474e-02,
        1.58495559e-02],
       [1.55824690e-03, 4.49343559e-01, 3.80466522e-01, 3.20473994e-01,
        2.68465816e-01, 1.47820393e-02, 1.83067282e-02, 2.19524066e-02,
        2.56580382e-02],
       [2.33454339e-03, 4.01314003e-01, 3.38262822e-01, 2.83568945e-01,
        2.36357124e-01, 2.20292741e-02, 2.72505932e-02, 3.26385747e-02,
        3.8

In [24]:
 max([1, 2, -1], 0)

TypeError: '>' not supported between instances of 'int' and 'list'

In [121]:
comb(3, -1)

0.0

In [77]:
runGame()


These are your cards: [11, 0, 7, 8]


AttributeError: type object 'StreamHandler' has no attribute 'level'

In [60]:
onetypeCall(np.array([0.837, 1.92, 0.837, 0.837, 1.916, 0.837, 0.837, 0.837, 0.837, 0.837, 0.837, 0.837, 0.837]), 4, 1, np.array([0 for i in range(13)]), 7)
    


0.2536748583718859

In [28]:
allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]
allCallProb

[array([0.82182144, 0.82182144, 0.82182144, 0.82182144, 0.82182144,
        0.82182144, 0.82182144, 0.82182144, 0.82182144]),
 array([0.48627712, 0.48627712, 0.48627712, 0.48627712, 0.48627712,
        0.48627712, 0.48627712, 0.48627712, 0.48627712]),
 array([[0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416, 0.2012416,
         0.2012416, 0.2012416, 0.2012416],
        [0.2012416, 0.2012416, 0.2012416

In [38]:
twotype(np.array([1, 2]), 4, 3, 2, np.array([3, 0]))


[[[0.5  0.5 ]
  [0.75 0.75]]

 [[0.5  0.75]
  [0.5  0.75]]]


array([[0.        , 0.        ],
       [0.20703125, 0.        ]])

In [30]:
np.array([1, 2])*np.array([2, 4])

array([2, 8])

In [79]:
pgrad(np.array([1.0 for i in range(13)]), 4, 2, 2, np.array([0 for i in range(13)]), 4, 0)/abs(pgrad(np.array([1.0 for i in range(13)]), 4, 2, 2, np.array([0 for i in range(13)]), 4, 0)).max()

array([ 1.        , -0.18181818, -0.18181818, -0.18181818,  1.        ,
       -0.18181818, -0.18181818, -0.18181818, -0.18181818, -0.18181818,
       -0.18181818, -0.18181818, -0.18181818])

In [78]:
onetypeCall(np.array([1, 1]), 1, 1, np.array([1, 1]), 1)

1.0

In [13]:
state

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [148]:
makeCall(np.array([2, 0, 8, 0]), np.array([0.76119419, 0.76119419, 0.76119419, 0.76119419, 0.76119419, 1.84452753,
 0.76119419, 0.76119419, 1.83708189, 1.84077897, 0.76119419, 0.76119419,
 0.76119419]), 4, np.array([0, 0, 0, 1, 0, 0,
 1, 2, 0, 0, 0, 0,
 0]), callsList)

(2, 13, 13)
(2, 13, 13)
(2, 13, 13)
2
2
2
2
2


[3, 0, 7, -1, 0.2423529805461931]

In [152]:
twotype(np.array([0.76119419, 0.76119419, 0.76119419, 0.76119419, 0.76119419, 1.84452753,
 0.76119419, 0.76119419, 1.83708189, 1.84077897, 0.76119419, 0.76119419,
 0.76119419]), 4, 2, 2, np.array([0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0,
 0]))

(2, 13, 13)


array([[0.00012375, 0.00012375, 0.00012375, 0.00012375, 0.00012375,
        0.00032281, 0.00012375, 0.00012375, 0.00032112, 0.00032195,
        0.00012375, 0.00012375, 0.00012375],
       [0.00012375, 0.00012375, 0.00012375, 0.00012375, 0.00012375,
        0.00032281, 0.00012375, 0.00012375, 0.00032112, 0.00032195,
        0.00012375, 0.00012375, 0.00012375],
       [0.00012375, 0.00012375, 0.00012375, 0.00012375, 0.00012375,
        0.00032281, 0.00012375, 0.00012375, 0.00032112, 0.00032195,
        0.00012375, 0.00012375, 0.00012375],
       [0.00012375, 0.00012375, 0.00012375, 0.00012375, 0.00012375,
        0.00032281, 0.00012375, 0.00012375, 0.00032112, 0.00032195,
        0.00012375, 0.00012375, 0.00012375],
       [0.00012375, 0.00012375, 0.00012375, 0.00012375, 0.00012375,
        0.00032281, 0.00012375, 0.00012375, 0.00032112, 0.00032195,
        0.00012375, 0.00012375, 0.00012375],
       [0.00032281, 0.00032281, 0.00032281, 0.00032281, 0.00032281,
        0.00084207, 0.00032

In [43]:
sum(np.array([0.76119419, 0.76119419, 0.76119419, 0.76119419, 0.76119419, 1.84452753,
 0.76119419, 0.76119419, 1.83708189, 1.84077897, 0.76119419, 0.76119419,
 0.76119419]))

13.13433029

In [57]:
np.array([[4,3]])*np.array([[2],[1]])

array([[8, 6],
       [4, 3]])

In [146]:
def tt(state, numcards, countOne, countTwo, ownCards):
    """ Output is a 13x13 matrix 
    state vector of size 13
    numCards, countOne, countTwo numbers
    ownCards vector of size 13
    
    Converts state to 2x13x13 matrix
    """
    prob = oneCardProbabilities(state)
    #probPairs = np.array(np.meshgrid(prob,prob)) # so that it has shape (2,13,13)
    probPairs = np.transpose(np.reshape(np.array(list(map(np.array,itertools.product(prob,prob)))),(state.size,state.size,2)),(2,0,1))
   
             #print(probPairs.shape)
#     probabilities = np.zeroes([len(state), len(state)])
#     for i in range(len(state)):
#         for j in range(len(state)):
#             twopairprob(probPairs[i, j], numcards)
    
    return ttp(probPairs, numcards, countOne, countTwo, ownCards)

def ttp(p, numcards, countCardOne, countCardTwo, ownCards):
    """ Output is a 13x13 matrix. 
    p.shape=2x13x13 >state made into grid representing pairs
    numcards, countCardOne, countCardTwo numbers
    ownCards vector of size 13
    
    Returns matrix representing probabilities of calls
    """
    prob = 0
    
    for i in range(countCardOne, 5):
        for j in range(countCardTwo, 5):
            prob = prob + (comb(numcards, i-ownCards) * (p[0])**(i-ownCards))
            
    print((comb(numcards, i-ownCards).reshape(ownCards.shape[0], 1) * (p[0])**(i-ownCards).reshape(ownCards.shape[0],1)).shape)
    
    return prob

In [147]:

tt(np.arange(13), 4, 2, 2, np.array([0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0,
 0]))

(13, 13)


array([[0.00218962, 0.00218962, 0.00218962, 0.00218962, 0.00218962,
        0.00218962, 0.00218962, 0.00218962, 0.00218962, 0.00218962,
        0.00218962, 0.00218962, 0.00218962],
       [0.0088227 , 0.0088227 , 0.0088227 , 0.0088227 , 0.0088227 ,
        0.0088227 , 0.0088227 , 0.0088227 , 0.0088227 , 0.0088227 ,
        0.0088227 , 0.0088227 , 0.0088227 ],
       [0.01999635, 0.01999635, 0.01999635, 0.01999635, 0.01999635,
        0.01999635, 0.01999635, 0.01999635, 0.01999635, 0.01999635,
        0.01999635, 0.01999635, 0.01999635],
       [0.03580876, 0.03580876, 0.03580876, 0.03580876, 0.03580876,
        0.03580876, 0.03580876, 0.03580876, 0.03580876, 0.03580876,
        0.03580876, 0.03580876, 0.03580876],
       [0.05635913, 0.05635913, 0.05635913, 0.05635913, 0.05635913,
        0.05635913, 0.05635913, 0.05635913, 0.05635913, 0.05635913,
        0.05635913, 0.05635913, 0.05635913],
       [0.08174774, 0.08174774, 0.08174774, 0.08174774, 0.08174774,
        0.08174774, 0.08174

In [93]:
np.arange(4).reshape(4,1)

array([[0],
       [1],
       [2],
       [3]])

In [107]:
np.array([[1, 2],[3, 4]])**np.reshape(np.array([1, 2]), (2, 1))

array([[ 1,  2],
       [ 9, 16]], dtype=int32)

In [114]:
np.tile(np.arange(4),4).reshape(4,4)**np.array([[1,2,3,4]])

array([[ 0,  1,  8, 81],
       [ 0,  1,  8, 81],
       [ 0,  1,  8, 81],
       [ 0,  1,  8, 81]], dtype=int32)

In [165]:
twotype(np.array([1, 1, 2]), 4, 2, 2, np.array([1, 0, 1]))

(2, 3, 3)


array([[0.51311953, 0.09329446, 0.63973344],
       [0.17230916, 0.03998334, 0.20991254],
       [0.63973344, 0.13994169, 0.78717201]])

In [18]:
def runGameTwo():
    bluffCalled = 0

    numPlayerCards = 4
    numCpuCards = 4
    deckSize = 13
    factor = 1
    
    
    deck = [i for i in range(deckSize) for j in range(4)]
    random.shuffle(deck)
    playerCards=deck[0:4]
    cpuCards = deck[4:8]
    #playerCards = [4, 8, 4, 9]
    #cpuCards = [1, 10, 2, 12]
    
    state = np.array([0.0 for i in range(deckSize)])
    ownCards = np.array([0 for i in range(deckSize)])
    #ownCards[6] = 1
    #ownCards[7] = 1
    #ownCards[9] = 1
    #ownCards[10] = 1
    
    for i in cpuCards:
        ownCards[i] = ownCards[i]+1
        
    print('These are your cards:', playerCards)
    #print(cpuCards)
    
    numrounds= 0
    while not bluffCalled:
        
        userIn = input('Make your decision:')
        if(userIn == "bluff"):
            bluffCalled = 1
            print(cpuCards)
            break
        else:
            userIn=userIn.split()
            call= np.array([int(userIn[0]), int(userIn[1]), int(userIn[2]), int(userIn[3])])
            
      
            
           
           
            
            
            if random.random()>0.5:
                print("Bluff")
                print(makeCall(call, state, numPlayerCards, ownCards, callsList))
                bluffCalled = 1
                print(cpuCards)
                
            else:
                print(makeCall(call, state, numPlayerCards, ownCards, callsList))
                
            
        
            
            

    

In [26]:
runGameTwo()

These are your cards: [6, 12, 0, 1]
[11, 7, 4, 7]


In [31]:
twotype(np.array([0.98159744, 0.98159744, 0.98159744, 0.98159744, 0.98159744, 0.98159744,
 0.98159744, 1.10785994, 0.98159744, 0.98159744, 1.10791171, 0.98159744,
 0.98159744]), 4, 2, 2, np.array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]))

array([[0.00020204, 0.00020204, 0.00648723, 0.00648723, 0.00020204,
        0.00020204, 0.00020204, 0.0002286 , 0.00648723, 0.00020204,
        0.00686424, 0.00020204, 0.00020204],
       [0.00020204, 0.00020204, 0.00648723, 0.00648723, 0.00020204,
        0.00020204, 0.00020204, 0.0002286 , 0.00648723, 0.00020204,
        0.00686424, 0.00020204, 0.00020204],
       [0.00176814, 0.00176814, 0.0594963 , 0.0594963 , 0.00176814,
        0.00176814, 0.00176814, 0.00200065, 0.0594963 , 0.00176814,
        0.06296168, 0.00176814, 0.00176814],
       [0.00176814, 0.00176814, 0.0594963 , 0.0594963 , 0.00176814,
        0.00176814, 0.00176814, 0.00200065, 0.0594963 , 0.00176814,
        0.06296168, 0.00176814, 0.00176814],
       [0.00020204, 0.00020204, 0.00648723, 0.00648723, 0.00020204,
        0.00020204, 0.00020204, 0.0002286 , 0.00648723, 0.00020204,
        0.00686424, 0.00020204, 0.00020204],
       [0.00020204, 0.00020204, 0.00648723, 0.00648723, 0.00020204,
        0.00020204, 0.00020

In [35]:
twotype(state, 4, 2, 2, ownCards)

array([[0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008],
       [0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008],
       [0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008],
       [0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008],
       [0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021008, 0.00021008],
       [0.00021008, 0.00021008, 0.00021008, 0.00021008, 0.00021008,
        0.00021008, 0.00021

In [47]:
1/2

0.5

In [45]:
def func(x):
    logger.info("etc")
    print(x**2)
    

In [46]:
func(4)

etc
16
